# Prototype-based Feature Importance

In [ ]:
# pip install -U scikit-fuzzy

In [39]:
import time
import numpy as np
import pandas as pd
import random
import shap
import pickle
from abc import ABC, abstractmethod 

import skfuzzy as fuzz
from scipy import linalg as la
from sklearn.metrics import accuracy_score
from sklearn.base import MultiOutputMixin, BaseEstimator
from sklearn.cluster import KMeans, SpectralClustering

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.6, style='whitegrid')
import skfuzzy as fuzz

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [40]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

### ClustererWrapper super class

In [41]:
class ClustererWrapper(MultiOutputMixin, BaseEstimator, ABC):

    def __init__(self, n_centers=3, seed=0):
        
        self.n_centers = n_centers
        self.centroids = None
        self.seed = seed
    
    @abstractmethod
    def fit(self, X_train, Y_train=None):
        pass

    @abstractmethod
    def predict(self, X):
        pass

    def f_importance(self, X):

        f_values = np.zeros(X.shape[1])
        for xi in enumerate(self.centroids):
            for xj in enumerate(self.centroids):
                f_values += np.abs(xi[1]-xj[1])
        
        return f_values/np.sum(f_values)
    


### Fuzzy c-means wrapper

In [82]:
class FuzzyCMeansWrapper(ClustererWrapper):

    def __init__(self, n_centers=3, seed=0):
           
        super().__init__(n_centers, seed)
  
    def fit(self, X_train, Y_train=None):
        
        self.centroids, _, _, _, _, _, _ = fuzz.cluster.cmeans(
        X_train.T, self.n_centers, m=2, error=0.005, maxiter=1000, seed=self.seed)
        
        return self
    
    def predict(self, X):

        u, _, _, _, _, _ = fuzz.cluster.cmeans_predict(
            X.T, self.centroids, m=2, error=0.005, maxiter=1000, seed=self.seed)

        return u.T
    

### K-means wrapper

In [83]:
class KMeansWrapper(ClustererWrapper):

    def __init__(self, n_centers=3, seed=0):

        super().__init__(n_centers, seed)
        self.kmeans = KMeans(self.n_centers, random_state=self.seed)
    
    def fit(self, X_train, Y_train=None):
        
        self.kmeans = self.kmeans.fit(X_train)
        self.centroids = self.kmeans.cluster_centers_

        return self

    def predict(self, X):

        X = np.array(X)
        prediction_matrix = np.zeros((len(X), len(self.centroids)))

        cluster_labels = self.kmeans.predict(X)
        for i, label in enumerate(cluster_labels):
            prediction_matrix[i, label] += 1
            
        return prediction_matrix

### Spectral clustering wrapper


```Note:```
Not sure if the method really makes sense for Spectral Clustering, as the method assumes that the clusters are spherical.

Another problem is that Sklearn's SpectralClustering class expects at least 2 point to perform the clustering. It would seem that ``shap.shap_values()`` function feeds the prediction function one sample at a time. 

Thus, the following error is thrown:  
<img src="spectral_clustering_error.png" width="100%">


In [88]:
class SpectralClusteringWrapper(ClustererWrapper):

    def __init__(self, n_centers=3, seed=0):

        super().__init__(n_centers, seed)
        self.spectral_clustering = SpectralClustering(self.n_centers, random_state=self.seed)

    def compute_centroids(self, X):
        
        centroids = np.zeros(self.n_centers)
        for k in range(self.n_centers):
            centroids[k] = np.mean([x for i, x in enumerate(X) if self.spectral_clustering.labels_[i] == k])

        return centroids

    def fit(self, X_train, Y_train=None):

        X_train = np.array(X_train)
        
        self.spectral_clustering = self.spectral_clustering.fit(X_train)
        self.centroids = self.compute_centroids(X_train)

        return self

    def predict(self, X):
        
        X = np.array(X)
        prediction_matrix = np.zeros((len(X), len(self.centroids)))

        cluster_labels = self.spectral_clustering.fit_predict(X)
        for i, label in enumerate(cluster_labels):
            prediction_matrix[i, label] += 1
            
        return prediction_matrix
        


### Simulation Pipeline

In [113]:
def pipeline(clustering_algorithm, df, dataset_name, n_centers):

    ALGORITHM_NAME = clustering_algorithm.__name__.replace('Wrapper', '')

    # computing the clustering ground truth
    clustering_algo = clustering_algorithm(n_centers, seed=0).fit(df.values)
    temp = clustering_algo.predict(df.values)
    y_true = np.argmax(clustering_algo.predict(df.values), axis=1)


    try:
        # load previously computed SHAP values if available
        shap_values = pickle.load(open('shap_values/'+dataset_name+'_'+ALGORITHM_NAME+'_shap.dat','rb'))

    except FileNotFoundError:
        # compute the SHAP values from SCRATCH
        explained_model = shap.KernelExplainer(clustering_algo.predict, df)
        shap_values = explained_model.shap_values(df)
        with open('shap_values/'+dataset_name+'_'+ALGORITHM_NAME+'_shap.dat', 'wb') as file:
            pickle.dump(shap_values, file)

    labels = ['c'+str(i) for i in range(1,n_centers+1)]
    shap.summary_plot(shap_values=shap_values, features=df.columns, feature_names=None, 
                  plot_type='bar', class_names=labels, color=plt.get_cmap("tab20c"), show=False)

    plt.xlabel("SHAP value", fontsize=18)
    plt.tick_params(axis='x', labelsize=18)
    plt.tick_params(axis='y', labelsize=18)
    plt.gca().patch.set_edgecolor('lightgrey')  
    plt.gca().patch.set_linewidth(1)
    plt.legend(fontsize=18)
    plt.savefig('shap_values/'+dataset_name+'_'+ALGORITHM_NAME+'_figure.pdf', bbox_inches='tight')
    plt.close()

    # computing the average SHAP values
    ave_shap_values = np.zeros(shap_values[0].shape[1])
    for shap_i in shap_values:
        ave_shap_values += np.mean(np.absolute(shap_i), axis=0)

    # sorting the features by their SHAP value
    shap_df = pd.DataFrame(columns=['features','shap'])
    shap_df['features'] = df.columns
    shap_df['shap'] = ave_shap_values
    shap_df.sort_values(by='shap', axis=0, ascending=False, inplace=True)
    shap_labels = shap_df['features'].tolist()

    errors_shap = [1.0]
    exclude = []

    # computing the perturbation errors for SHAP
    for fi in shap_labels:

        df_temp = df.copy()
        exclude.append(fi)

        for column in exclude:
            df_temp[column] = df_temp[column].mean()

        y_pred = np.argmax(clustering_algo.predict(df_temp.values), axis=1)
        errors_shap.append(accuracy_score(y_true, y_pred))
    
    # computing the results of the PBFI method
    pbfi_df = pd.DataFrame(columns=['features','pfi'])
    pbfi_df['features'] = df.columns
    pbfi_df['pfi'] = clustering_algo.f_importance(df.values)
    pbfi_df.sort_values(by='pfi', axis=0, ascending=False, inplace=True)
    pbfi_labels = pbfi_df['features'].tolist()

    errors_pbfi = [1.0]
    exclude = []

    # computing the perturbation errors for PFI
    for fi in pbfi_labels:

        df_temp = df.copy()
        exclude.append(fi)

        for column in exclude:
            df_temp[column] = df_temp[column].mean()

        y_pred = np.argmax(clustering_algo.predict(df_temp.values), axis=1)
        errors_pbfi.append(accuracy_score(y_true, y_pred))


    sns.lineplot(x = range(1,len(shap_labels)+2), y=errors_shap, marker='o', markersize=10, label = "SHAP");
    sns.lineplot(x = range(1,len(pbfi_labels)+2), y=errors_pbfi, marker='D', markersize=10, label = "PBFI");
    plt.tick_params(axis='x', labelsize=18)
    plt.tick_params(axis='y', labelsize=18)
    plt.xlabel('rank', fontsize=18)
    plt.ylabel('accuracy', fontsize=18)

    plt.gca().fill_between(range(1,len(shap_labels)+2), errors_shap, errors_pbfi, alpha=0.2, color='grey')

    def label_points(shap_labels, errors_shap, pbfi_labels, errors_pbfi, ax):
        x = range(1,len(shap_labels)+2)
        shap_points = pd.DataFrame({'x': x, 'y': errors_shap, 'label': [''] + shap_labels})
        pbfi_points = pd.DataFrame({'x': x, 'y': errors_pbfi, 'label': [''] + pbfi_labels})

        for point_shap, point_pbfi in zip(shap_points.iterrows(), pbfi_points.iterrows()):

            if point_shap[1]['y'] != point_pbfi[1]['y']:
                ax.text(point_shap[1]['x']+.02, point_shap[1]['y']+.02, str(point_shap[1]['label']), size=11)
                ax.text(point_pbfi[1]['x']+.02, point_pbfi[1]['y']+.02, str(point_pbfi[1]['label']), size=11)
            elif point_shap[1]['label'] != point_pbfi[1]['label']:
                ax.text(point_shap[1]['x']+.02, point_shap[1]['y']+.06, str(point_shap[1]['label']), size=11, color='#1f77b4')
                ax.text(point_pbfi[1]['x']+.02, point_pbfi[1]['y']+.02, str(point_pbfi[1]['label']), size=11, color='#ff7f0e')
            else:
                ax.text(point_shap[1]['x']+.02, point_shap[1]['y']+.02, str(point_shap[1]['label']), size=11)
            

    label_points(shap_labels, errors_shap, pbfi_labels, errors_pbfi, plt.gca())

    plt.savefig('feature_importance/error_'+dataset_name+'_'+ALGORITHM_NAME+'.pdf', bbox_inches='tight')
    plt.close()

### Running the simulations

In [116]:
datasets = ['ecoli', 'glass', 'heart-statlog', 'iris', 'liver-disorders', 'pima', 'vehicle', 
            'yeast', 'vertebra-column-2c', 'saheart', 'new-thyroid',
            'echocardiogram', 'appendicitis', 'hayes-roth']

'''
'wine-quality-red',
'''

sns.set_style('whitegrid')
paper_rc = {'lines.linewidth': 1, 'lines.markersize': 7} 
sns.set_context('paper', font_scale=1.8, rc=paper_rc)

for dataset in datasets:
    
    # loading the current dataset
    df = pd.read_csv('datasets/'+dataset+'.csv')
    n_centers = len(np.unique(df.values[:,-1]))
    df = df.drop(df.columns[-1], axis='columns')

    for clustering_algorithm in [FuzzyCMeansWrapper, KMeansWrapper]:#, SpectralClusteringWrapper]:

        pipeline(clustering_algorithm=clustering_algorithm, df=df, dataset_name=dataset, n_centers=n_centers)

Using 846 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 846/846 [2:39:28<00:00, 11.31s/it]    
Using 846 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 846/846 [1:47:21<00:00,  7.61s/it]
Using 1599 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 1599/1599 [7:17:24<00:00, 16.41s/it]  
Using 1599 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  8%|▊         | 120/1599 [26:16<5:23:49, 13.14s/it]


KeyboardInterrupt: 